speech_to_text_en >>

In [1]:
import os
from moviepy.editor import VideoFileClip
import warnings
import torch
from transformers import pipeline

# ----------------
# Suppress warnings
# ----------------
warnings.filterwarnings("ignore")

# ----------------
# Device & dtype
# ----------------
_DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
_DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

# ----------------
# Load model ONCE
# ----------------
print("🎙️ Loading Whisper STT model...")
_stt_pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v3",
    torch_dtype=_DTYPE,
    device=_DEVICE,
)
print("✅ Whisper model loaded")


def speech_to_text_en(audio_path: str, language: str = "english") -> str:
    """
    ดึงเสียงจากวิดีโอและแปลงเป็นข้อความ โดยใช้ไฟล์ชั่วคราวเพื่อเลี่ยง TypeError
    """
    temp_audio = "temp_whisper_input.wav"
    
    try:
        # 1. เช็คว่าเป็นไฟล์วิดีโอหรือไม่
        if audio_path.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            print(f"🎬 Extracting audio from: {audio_path}")
            video = VideoFileClip(audio_path)
            
            # ตรวจสอบว่าวิดีโอมีเสียงจริงไหม
            if video.audio is None:
                video.close()
                return "" # หรือจะ raise Error ตามที่คุณต้องการ
            
            # เขียนไฟล์เสียงออกมาเป็น .wav (16kHz, Mono ตามที่ Whisper ชอบ)
            # วิธีนี้จะเลี่ยง TypeError: arrays to stack... ได้ 100%
            video.audio.write_audiofile(
                temp_audio, 
                fps=16000, 
                nbytes=2, 
                codec='pcm_s16le', 
                verbose=False, 
                logger=None
            )
            video.close()
            path_to_process = temp_audio
        else:
            path_to_process = audio_path

        # 2. ส่ง Path ให้ Pipeline (Whisper จะเปิดไฟล์เองด้วย FFmpeg)
        print("🎙️ Transcribing...")
        result = _stt_pipe(
            path_to_process,
            chunk_length_s=30,
            generate_kwargs={"language": "english","return_timestamps": True}
        )

        # 3. ลบไฟล์ชั่วคราว
        if os.path.exists(temp_audio):
            os.remove(temp_audio)

        return result["text"]

    except Exception as e:
        # กรณี Error ก็ต้องลบไฟล์ทิ้งด้วย
        if os.path.exists(temp_audio):
            os.remove(temp_audio)
        print(f"❌ Error during STT: {e}")
        raise e

🎙️ Loading Whisper STT model...


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


✅ Whisper model loaded


text_translation >>

In [2]:
"""แปล Text ภาษาอังกฤษเป็น Text ภาษาไทย"""

"""แปลง Text ไทย เป็นเสียงไทย"""

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#โหลด Model
model_id = "scb10x/typhoon-translate-4b"

# โหลด Tokenizer และ Model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
model_id, 
torch_dtype=torch.bfloat16, 
device_map={"": 0}, #บังคับใช้ GPU
)
  
def text_translation(ENG_text_path):
    #ข้อความที่ต้องการแปล
    text = ENG_text_path


    # สำคัญ: ต้องใช้ System Prompt ตามที่โมเดลกำหนดเพื่อให้ผลลัพธ์แม่นยำ
    messages3 = [
        {"role": "system", "content": "Translate the following text into Thai."},
        {"role": "user", "content": text},
    ]

    # เตรียม Input
    input_ids = tokenizer.apply_chat_template(
    messages3, 
    add_generation_prompt=True, 
    return_tensors="pt"
    ).to(model.device)

    
        # สั่งให้โมเดล Generate ผลลัพธ์
    outputs = model.generate(
        input_ids, 
        max_new_tokens=512, 
        do_sample=False, # แนะนำให้ปิด sampling เพื่อความแม่นยำในการแปล
        temperature=None, # ล้างค่าที่ไม่จำเป็นออก
        top_p=None
    )
    TH_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
    return TH_text


`torch_dtype` is deprecated! Use `dtype` instead!
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.42s/it]


text_to_speech_TH >>

In [3]:
# -----------------------------
# MMS-TTS (Standard TTS)
# -----------------------------
import torch
import scipy.io.wavfile as wavfile
from transformers import VitsModel, AutoTokenizer

print("🔊 Loading MMS-TTS-THAI model...")
_mms_model_name = "facebook/mms-tts-tha"
_mms_model = VitsModel.from_pretrained(_mms_model_name)
_mms_tokenizer = AutoTokenizer.from_pretrained(_mms_model_name)
print("✅ MMS-TTS model loaded")



def text_to_speech_TH(text: str, output_path: str = "thai_mms.wav"):
    try:
        inputs = _mms_tokenizer(text, return_tensors="pt")

        with torch.no_grad():
            waveform = _mms_model(**inputs).waveform

        sample_rate = _mms_model.config.sampling_rate

        wavfile.write(
            output_path,
            rate=sample_rate,
            data=waveform[0].cpu().numpy(),
        )
        # ไม่ต้อง return dictionary แล้ว แค่พิมพ์บอกสถานะ (หรือไม่ต้องพิมพ์ก็ได้)
        print(f"✅ บันทึกเสียงภาษาไทยสำเร็จที่: {output_path}")

    except Exception as e:
        # หากเกิด Error ก็ให้พิมพ์แจ้งเตือนแทนการส่งค่ากลับ
        print(f"❌ เกิดข้อผิดพลาด: {e}")

🔊 Loading MMS-TTS-THAI model...
✅ MMS-TTS model loaded


In [4]:
import os
from moviepy.editor import VideoFileClip, AudioFileClip
from moviepy.audio.fx.all import audio_loop

def video_sound_editor(new_sound_path, video_path, output_path):
    """
    รวมไฟล์เสียงใหม่เข้ากับวิดีโอเดิม 
    - ถ้าเสียงสั้นกว่า: จะวนลูปเสียง (Loop)
    - ถ้าเสียงยาวกว่า: จะตัดเสียงให้พอดี (Trim)
    """
    try:
        # 1. โหลดคลิปต้นทาง
        video = VideoFileClip(video_path)
        new_audio = AudioFileClip(new_sound_path)

        # 2. จัดการความยาวไฟล์เสียงให้เท่ากับวิดีโอ
        if new_audio.duration < video.duration:
            # กรณีเสียงสั้นกว่า: ให้วนลูปเสียงจนจบวิดีโอ
            print(f"🔄 Audio is shorter than video. Looping audio...")
            final_audio = audio_loop(new_audio, duration=video.duration)
        else:
            # กรณีเสียงยาวกว่า: ตัดให้พอดีกับวิดีโอ
            print(f"✂️ Audio is longer than video. Trimming audio...")
            final_audio = new_audio.subclip(0, video.duration)

        # 3. รวมเสียงใหม่เข้ากับวิดีโอเดิม
        final_video = video.set_audio(final_audio)

        # 4. บันทึกไฟล์ผลลัพธ์
        print(f"💾 Exporting video to: {output_path}")
        final_video.write_videofile(
            output_path,
            fps=video.fps,           # ใช้ค่า fps เดิมของวิดีโอ
            codec="libx264",         # มาตรฐานสำหรับวิดีโอ
            audio_codec="aac",       # มาตรฐานสำหรับเสียง
            audio=True,              # บังคับให้เขียนแทร็กเสียง
            temp_audiofile='temp-audio-final.m4a',
            remove_temp=True
        )

        # 5. ปิด Resource (สำคัญมากเพื่อไม่ให้ไฟล์ถูก Lock)
        video.close()
        new_audio.close()
        final_audio.close()
        final_video.close()
        
        print("✅ Video processing completed successfully!")

    except Exception as e:
        print(f"❌ Error in video_sound_editor: {e}")
        # พยายามปิด Resource แม้จะเกิด Error
        if 'video' in locals(): video.close()
        if 'new_audio' in locals(): new_audio.close()

In [6]:
import customtkinter as ctk
from tkinter import filedialog
import os

# ตั้งค่า Theme ให้ดูทันสมัย
ctk.set_appearance_mode("System")  # ปรับตาม Windows (Light/Dark)
ctk.set_default_color_theme("blue")

def show_modern_msg(title, message):
    """สร้างหน้าต่างแจ้งเตือนแบบ Modern"""
    msg_window = ctk.CTkToplevel()
    msg_window.title(title)
    msg_window.geometry("350150")
    msg_window.attributes('-topmost', True)
    
    # วางตำแหน่งให้กลางจอ
    msg_window.update_idletasks()
    x = (msg_window.winfo_screenwidth() // 2) - (msg_window.winfo_width() // 2)
    y = (msg_window.winfo_screenheight() // 2) - (msg_window.winfo_height() // 2)
    msg_window.geometry(f"+{x}+{y}")

    label = ctk.CTkLabel(msg_window, text=message, font=("Leelawadee UI", 14), wraplength=300)
    label.pack(expand=True, padx=20, pady=20)

    btn = ctk.CTkButton(msg_window, text="ตกลง", command=msg_window.destroy, width=100)
    btn.pack(pady=(0, 20))
    
    # บังคับให้ User กดตกลงก่อนไปต่อ (Optional)
    msg_window.grab_set() 
    msg_window.wait_window()

def processing_pipline():
    root = ctk.CTk()
    root.withdraw()

    # 1. แจ้งเตือนขั้นแรก
    show_modern_msg("เตรียมพร้อม", "กรุณาเลือกไฟล์วิดีโอต้นฉบับครับ ✨")
    
    original_video_path = filedialog.askopenfilename(
        title="เลือกไฟล์วิดีโอต้นฉบับ",
        filetypes=[("Video", "*.mp4 *.avi *.mkv")]
    )

    if not original_video_path: return

    # [ ... ส่วนประมวลผลเหมือนเดิม ... ]
    base_dir = os.path.dirname(original_video_path)
    sound_path = os.path.join(base_dir, "translated_audio.wav")
    
    # ประมวลผล
    eng_text = speech_to_text_en(original_video_path)
    th_text = text_translation(eng_text)
    text_to_speech_TH(th_text, sound_path)

    # 2. แจ้งเตือนขั้นที่สอง (นุ่มนวล)
    show_modern_msg("ประมวลผลเสร็จแล้ว", "เราเตรียมเสียงพากย์เสร็จแล้วครับ\nขั้นตอนสุดท้าย: เลือกที่เก็บไฟล์วิดีโอ")

    final_output_path = filedialog.asksaveasfilename(
        title="บันทึกผลลัพธ์",
        defaultextension=".mp4",
        initialfile="video_TH.mp4"
    )

    if final_output_path:
        video_sound_editor(sound_path, original_video_path, final_output_path)
        show_modern_msg("สำเร็จ!", "สร้างวิดีโอพากย์ไทยเรียบร้อยแล้วครับ 🎉")
        
        if os.path.exists(sound_path):
            os.remove(sound_path)

    root.destroy()

In [7]:
processing_pipline()

🎬 Extracting audio from: C:/Users/Napat/Downloads/danmachi.mp4


Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


🎙️ Transcribing...


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ บันทึกเสียงภาษาไทยสำเร็จที่: C:/Users/Napat/Downloads\translated_audio.wav
✂️ Audio is longer than video. Trimming audio...
💾 Exporting video to: C:/Users/Napat/Downloads/video_TH.mp4
Moviepy - Building video C:/Users/Napat/Downloads/video_TH.mp4.
MoviePy - Writing audio in temp-audio-final.m4a


MoviePy - Done.
Moviepy - Writing video C:/Users/Napat/Downloads/video_TH.mp4



Moviepy - Done !
Moviepy - video ready C:/Users/Napat/Downloads/video_TH.mp4
✅ Video processing completed successfully!
